In [1]:
import sympy as sp
from sympy import *
import numpy as np

Experimentos realizados de forma empírica geralmente retornam dados semelhantes a pontos discretos. O problema nesse
tipo de situação é fato de não dispor de uma expressão analítica que permita descobrir determinado valor de retorno
para qualquer ponto arbitrário da variável em estudo. Para estas situações são utilizados recursos matemáticos que 
permitem determinar um comportamento aproximado da variação desse conjunto de dados e assim conseguir prever de forma
satisfatória quais serão os resultados nesses pontos arbitrários. Um desses recursos, amplamente utilizado nesse tipo
de problema é a utilização de funções interpoladoras. Essa técnica basicamente une os pontos conhecidos através de uma 
curva e essa curva passa a ser a função representativa da Lei de Variação dessa grandeza dentro desse intervalo. A 
relação para a correspondência entre as variáveis e o valor da grandeza constitui o modelo matemático.

Para que as previsões do fenômeno possam ser satisfatórias é necessário que o modelo matemático seja escolhido de forma
adequada, sendo essa escolha uma etapa fundamental para o êxito do processo.

Essas técnicas de interpolação são um dos pilares do Método dos Elementos Finitos. Como neste método o ponto principal 
não é conhecimento dos infinitos pontos (pela dificuldade de determinação dessas equações) mas sim somente de uma 
quantidade dita como suficiente para descrever o comportamento da estrutura em análise, a interpolação terá a função
de descrever o comportamento dos pontos que estão entre os nós do modelo.

Neste caso, essas Funções de Interpolação devem ser associadas a uma outra condição muito importante no estudo de 
estruturas: a física do material. A deformação de um elemento está diretamente associada a carga que este recebe e sua
rigidez.

Essa condição adicional será descrita através do conceito de trabalho e energia de deformação.

[Aqui talvez colocar um link de um exemplo simples de determinação da rigidez de uma mola pelo método de 
conservação de energia e mais teoria da parte física]

Nesse interim, o que o método traz é que a energia externa introduzida no elemento através dos carregamentos
é transformada em energia de deformação, a qual está associada a sua configuração deformada. Essa configuração 
deformada por sua vez está descrita pela função de interpolação. Desta forma, calculando a energia de deformação e 
igualando esta ao trabalho externo podemos determinar, em caráter aproximado, a rigidez do elemento finito.


- Formulação da Função:

In [2]:
# Definição das variáveis
x, n11, n12, n13, n14, u1, u2, C1, C2, C3, C4, l, M = sp.symbols('x, n11, n12, n13, n14, u1, u2, C1, C2, C3, C4, l, M')
Desloc1, B, Curvatura, VetorDesloc = sp.symbols('v1, B, v2l, VetorDesloc')
v1, v2, v1linha, v2linha, f1, M1, f2, M2, Delta, Coef= sp.symbols('v1, v2, v1linha, v2linha, f1, M1, f2, M2, Delta, Coef')
dteta, dx = sp.symbols('dteta, dx')

In [5]:
# Escolha da função interpoladora para os deslocamentos conforme seu GL
v = C1 + C2*x + C3*x**2 + C4*x**3

    # Representando a função de forma matricial

hx1 = sp.linear_eq_to_matrix([v], [C1, C2, C3, C4])[0]
hx = sp.Matrix(hx1).values()    # Matriz que contém as variáveis
C = np.array([(C1),(C2), (C3), (C4)])   # Matriz coluna dos coeficientes desconhecidos

# Função de inclinação da viga
Inclinacao = diff(v, x)

- Cálculo dos coeficientes relacionando os deslocamentos dentro do elemento com os deslocamentos nodais

In [ ]:
# Cálculo dos coeficientes desconhecidos da função interpoladora

'Para x = 0 (nó 1) v = v1 e θ = v1linha'

    # V1 - Deslocamento no nó 1
Desloc1 = v.subs(x, 0)

    # V1' - Inclinação no nó 1
Inclinacao_p1 = Inclinacao.subs(x, 0)


'Para x = L (nó 2) v = v2 e θ = v2linha'

    # V2
Desloc2 = v.subs(x, l)
# sp.pprint(v2)

    # V2'
Inclinacao_p2 = Inclinacao.subs(x, l)
# sp.pprint(v22)

'Matricialmente temos:'
A1 = sp.linear_eq_to_matrix([Desloc1], [C1, C2, C3, C4])
A2 = sp.linear_eq_to_matrix([Inclinacao_p1], [C1, C2, C3, C4])
A3 = sp.linear_eq_to_matrix([Desloc2], [C1, C2, C3, C4])
A4 = sp.linear_eq_to_matrix([Inclinacao_p2], [C1, C2, C3, C4])

A = np.array([(A1[0]), (A2[0]), (A3[0]), (A4[0])])  # Matriz geometria do elemento

Ainv = sp.Matrix(A).inv()   # Inversa da matriz geometria

Delta = np.dot(A, C)    # Matriz Deslocamentos nodais = Matriz Geometria do elemento x Matriz dos coeficientes

'Como os deslocamentos são representados pela matriz Hx multiplicada pela Matriz de Coeficientes C:'

nx = np.dot(hx, Ainv)   # Matriz Função de Forma do Elemento Finito


expn11 = sp.Eq(n11, nx[0])
expn12 = sp.Eq(n12, nx[1])
expn13 = sp.Eq(n13, nx[2])
expn14 = sp.Eq(n14, nx[3])

sp.pprint(expn11)
sp.pprint(expn12)
sp.pprint(expn13)
sp.pprint(expn14)


In [ ]:
# Deformações internas no elemento a partir dos deslocamentos nodais

'As deformações por flexão estão associadas à curvatura apresentada pela viga ao longo de seu comprimento'

v2linhas = diff(Inclinacao, x)   # v2linhas = Curvatura

B0 = sp.linear_eq_to_matrix([v2linhas], [C1, C2, C3, C4])[0]

B = B0 * Ainv

print(B)

VetorDesloc = np.array([(v1), (v1linha), (v2), (v2linha)])

Curvatura = B * VetorDesloc     # Representação de v2linhas na forma matricial


VetorDeslocT = sp.Matrix(VetorDesloc).transpose()

ForcNodal = np.array([(f1), (M1), (f2), (M2)])

O trabalho externo é dado pela multiplicação das forças e seus correspondentes deslocamentos juntamente com os momentos 
e seus correspondentes ângulos

Nessa etapa o VetorDesloc foi transposto para que a multiplicação pudesse ser feita 

In [ ]:
Trabalho_Externo = np.dot(VetorDeslocT, ForcNodal)

Para o trabalho interno levamos em consideração a deformação da viga por flexão associada à curvatura. Aliada a 
condição externa e a curvatura virtual impostas podemos contabilizar o trabalho interno decorrente da ação do momento
fletor e o ângulo que este define em toda viga

In [ ]:
def trabalho_interno():
    return Integral(M(dteta, 0, l))

Da teoria de vigas substituímos o dteta da integral

In [ ]:
Tinterno = Integral(sp.Matrix(Curvatura).transpose()*M, (x, 0, l))

... Continua